In [ ]:

# Default locations if parameter not passed
benchmark_data_path = "/scratch/rodrigo.freitas/remy-schedulers/big_matrix.pkl"


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from rich import print as rprint
import seaborn as sns
import os
from scipy.stats import bootstrap

benchmark_data = pickle.load(open(benchmark_data_path, "rb"))

df = pd.DataFrame(benchmark_data["dataframe"])

# create the output folder if it does not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


rprint(df.columns)
rprint(df.head())
rprint(df.tail())

Index(['name', 'sim_deviation', 'sim_rr', 'sim_device_number', 'full_output',
       'sim_task_number', 'time'],
      dtype='object')

name  sim_deviation  sim_rr  sim_device_number  \
0  time-measure            NaN       2                 10   
1  time-measure            NaN       2                 10   
2  time-measure            NaN       2                100   
3  time-measure            NaN       2                100   
4  time-measure            NaN       2                500   

                                         full_output  sim_task_number  \
0  [1749571026.293480] [sorgan:3140029:0]        ...              100   
1  [1749571026.830748] [sorgan:3140095:0]        ...              100   
2  [1749571027.375037] [sorgan:3140149:0]        ...              100   
3  [1749571027.928369] [sorgan:3140211:0]        ...              100   
4  [1749571028.484353] [sorgan:3140268:0]        ...              100   

       time  
0  0.538142  
1  0.532958  
2  0.547304  
3  0.549388  
4  0.599597

name  sim_deviation  sim_rr  sim_device_number  \
83  dist-measure            0.4      16                 20   
84  dist-measure            0.6      16                 20   
85  dist-measure            0.6      16                 20   
86  dist-measure            0.8      16                 20   
87  dist-measure            0.8      16                 20   

                                          full_output  sim_task_number  \
83  [1749571278.029374] [sorgan:3146434:0]        ...              500   
84  [1749571278.579127] [sorgan:3146485:0]        ...              500   
85  [1749571279.131130] [sorgan:3146540:0]        ...              500   
86  [1749571279.683482] [sorgan:3146593:0]        ...              500   
87  [1749571280.234563] [sorgan:3146645:0]        ...              500   

        time  
83  0.543234  
84  0.543810  
85  0.546626  
86  0.543518  
87  0.547049

In [9]:
sns.set_theme()

colors = [
    "#7f7f7f",  # grey
    "#621dac",  # main purple
    "#c5702d",  # orange
    "#000000",  # black,
    "#099892",  # teal
    "#ffd400",  # yellow
    "#7e57c4",  # pink/purple,
]
colors_backup = colors.copy()

In [10]:
def calculate_bootstrap_error(data):
    n_runs = benchmark_data["config"].metadata.runs
    if n_runs < 2:
        return 0
    res = bootstrap(
        (data,), np.mean, confidence_level=0.95, n_resamples=1000, method="basic"
    )
    return res.standard_error

In [11]:
bench_names = df["name"].unique()
run_metadata = benchmark_data["config"].applications

for bench in bench_names:
    bench_metadata = run_metadata[bench]
    n_runs = benchmark_data["config"].metadata.runs
    print(f"Benchmark: {bench} - Runs: {n_runs}")
    # print(bench_metadata)
    df_bench = df[df["name"] == bench]
    for idx, row in df_bench.iterrows():
        print(f"--- Row {idx} ---")
        for col, val in row.items():
            print(f"{col}: {val}")   # printing val directly never truncates
        print()




Benchmark: time-measure - Runs: 2
--- Row 0 ---
name: time-measure
sim_deviation: nan
sim_rr: 2
sim_device_number: 10
full_output: [1749571026.293480] [sorgan:3140029:0]        ib_verbs.h:87   UCX  ERROR ibv_exp_query_device(mlx4_0) returned 95: Operation not supported
Time Simulation (microseconds) : 98 
Distance : 0
Norm rCC:-nan
Distance : 4.93125e-15
Norm r2CC:-nan

libibverbs: Warning: no userspace device-specific driver found for /sys/class/infiniband_verbs/uverbs1
libibverbs: Warning: no userspace device-specific driver found for /sys/class/infiniband_verbs/uverbs0

sim_task_number: 100
time: 0.538142123259604

--- Row 1 ---
name: time-measure
sim_deviation: nan
sim_rr: 2
sim_device_number: 10
full_output: [1749571026.830748] [sorgan:3140095:0]        ib_verbs.h:87   UCX  ERROR ibv_exp_query_device(mlx4_0) returned 95: Operation not supported
Time Simulation (microseconds) : 96 
Distance : 0
Norm rCC:-nan
Distance : 4.2317e-15
Norm r2CC:-nan

libibverbs: Warning: no userspace de